In [2]:
!pip -q install datasets transformers sentence-transformers accelerate gradio

In [3]:
import math
from datetime import datetime
import wandb 

from datasets import Dataset
from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader

# Config hyperparameters
DATA_PATH = "/kaggle/input/clear-data/clear_data.jsonl"
BASE_MODEL = "intfloat/multilingual-e5-base"
EPOCHS = 1
LEARNING_RATE = 2e-5
BATCH_SIZE = 8
SAVE_PATH = (
    "//kaggle/working/output/"
    + BASE_MODEL.replace("/", "-")
    + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
for x in [DATA_PATH, BASE_MODEL, EPOCHS, LEARNING_RATE, BATCH_SIZE, SAVE_PATH]:
    print(x)



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/clear-data/clear_data.jsonl
intfloat/multilingual-e5-base
1
2e-05
8
//kaggle/working/output/intfloat-multilingual-e5-base2023-09-11_07-27-14


In [7]:
# train Model
# Load model
word_embedding_model = models.Transformer(BASE_MODEL)
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Load data
train_samples = []
origin_ds = Dataset.from_json(DATA_PATH).train_test_split(test_size=0.1, seed=42)
for row in origin_ds["train"]:
    train_samples.append(InputExample(texts=[row["user_question"], row["question"]]))

print("Train samples:", len(train_samples))

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=BATCH_SIZE)

# Config loss function
train_loss = losses.MultipleNegativesRankingLoss(model)

# save log to file 
             
# def log_callback_st(scores, num_steps, epoch, steps):
#     wandb.log(
#         {
#             "epoch": epoch,
#             "steps": steps,
#             "train_loss": scores[0],
#         }
#     )
#     # save log to file
#     with open(f"{SAVE_PATH}/log.txt", "a") as f:
#         f.write(
#             f"epoch: {epoch}, steps: {steps}, train_loss: {scores[0]}\n"
#         )


# Train the model and show loss scores per epoch
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=EPOCHS,
    use_amp=True,
    warmup_steps=math.ceil(len(train_dataloader) * EPOCHS * 0.1),
    output_path=SAVE_PATH,
    optimizer_params={"lr": LEARNING_RATE},
    # callback=log_callback_st,
    # evaluation_steps=len(train_dataloader)*EPOCHS//1000,
    # show_progress_bar=True,
    
)

Train samples: 117874


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/14735 [00:00<?, ?it/s]

In [8]:
from huggingface_hub import login
login()

In [9]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_folder(repo_id="ThanhRise/intfloat-multilingual-e5-base2023-08-30_16-52-03",
                 folder_path="/kaggle/working/output/intfloat-multilingual-e5-base2023-09-11_02-31-04")

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

'https://huggingface.co/ThanhRise/intfloat-multilingual-e5-base2023-08-30_16-52-03/tree/main/'

In [4]:
data = Dataset.from_json(DATA_PATH)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0769a52606229b23/0.0.0. Subsequent calls will reuse this data.


In [5]:
data

Dataset({
    features: ['url', 'title', 'user_question', 'question', 'answer', 'citation'],
    num_rows: 130972
})

*   <font size = 5 color = 'Green'> Test model with test data </font>

In [10]:
test_ds = Dataset.from_json(DATA_PATH).train_test_split(test_size=0.1, seed=42)['test']

In [52]:
test_ds

Dataset({
    features: ['url', 'title', 'user_question', 'question', 'answer', 'citation'],
    num_rows: 13098
})

In [8]:
model = SentenceTransformer('ThanhRise/intfloat-multilingual-e5-base2023-08-30_16-52-03')
# show loss scores in train process

In [14]:
corpus = test_ds['question']
corpus_embeddings = model.encode(corpus)

Batches:   0%|          | 0/410 [00:00<?, ?it/s]

In [29]:
# data.shuffle()[0]['question']

'Phó Viện trưởng Viện Kiểm sát nhân dân tối cao thuộc biên chế nào? Được biết Viện Kiểm sát nhân dân tối cao là cơ quan ở trung ương. Cho tôi hỏi hỏi biên chế các chức danh ở cơ quan này có khác gì so với địa phương không? Cụ thể chức danh Phó Viện trưởng Viện Kiểm sát nhân dân tối cao thuộc biên chế nào? Xin cảm ơn!'

In [62]:
origin_ds['train'].shuffle()[0]['user_question'] 

'Mức độ sử dụng sản phẩm, dịch vụ của thành viên, hợp tác xã thành viên là gì?'

In [76]:
test_ds.shuffle()[0]['user_question']

'Làm chết người trong lúc tự vệ thì truy cứu trách nhiệm hình sự như thế nào? Quy định các tình tiết giảm nhẹ trách nhiệm hình sự? Quy định về tội cố ý gây thương tích hoặc gây tổn hại cho sức khỏe của người khác?\nKhi tôi đi làm về tôi có bị một nhóm 3 người chặn đường đánh tới tấp, do không chịu đựng nổi tôi đã đánh trả đối phương nhằm khiến họ đau và dừng viêc đánh đập tôi. Tuy nhiên, vì sự đánh trả của tôi mà một người trong số đó đã bị mất đà và ngã đập đầu xuống nền nhà mà chết. Vậy tôi sẽ bị truy cứu về tội gì ạ? Tôi có được hưởng tình tiết giảm nhẹ không? \nCâu hỏi của anh Thanh (Hải Phòng)'

In [96]:
example = test_ds.shuffle()
print(example[0]['user_question'])
print()
print(example[0]['question'])

Mình đọc báo thì thấy từ tháng 7 lương cơ sở sẽ tăng 1490000 ngàn đồng/tháng. Vậy mình muốn hỏi khi lương cơ sở tăng thì tiền thưởng danh hiệu Chiến sĩ thi đua cấp tỉnh sẽ là bao nhiêu nhỉ? Mình cảm ơn!

Mức tiền thưởng danh hiệu Chiến sĩ thi đua cấp tỉnh 2019


In [17]:
from sentence_transformers import util
def get_similarity(question, topk=3):
    q_embeddings = model.encode(question)
    result = util.semantic_search(q_embeddings,corpus_embeddings)
    result = [(x['corpus_id'],x['score']) for x in result[0]]
    return [(corpus[i],test_ds[i]["answer"],score) for i, score in result]

In [97]:
q = "Mình đọc báo thì thấy từ tháng 7 lương cơ sở sẽ tăng 1490000 ngàn đồng/tháng. Vậy mình muốn hỏi khi lương cơ sở tăng thì tiền thưởng danh hiệu Chiến sĩ thi đua cấp tỉnh sẽ là bao nhiêu nhỉ? Mình cảm ơn!"
for a,b,c in get_similarity(q):
    print(f"Text: {a}\n\nAnswer: {b}\n\nScore: {c}")
    print('-----'*20)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Text: Mức tiền thưởng danh hiệu Chiến sĩ thi đua cấp tỉnh 2019

Answer: Căn cứ pháp lý:
Nghị định 91/2017/NĐ-CP
Nghị định 38/2019/NĐ-CP
Từ ngày 01 tháng 7 năm 2019, mức lương cơ sở là 1.490.000 đồng/tháng.
Theo Điều 69 Nghị định 91/2017/NĐ-CP thì: Mức tiền thưởng danh hiệu Danh hiệu Chiến sĩ thi đua cấp bộ, ban, ngành, tỉnh, đoàn thể trung ương được tặng Bằng chứng nhận, huy hiệu và được thưởng 3,0 lần mức lương cơ sở.
Như vậy, từ ngày 01 tháng 7 năm 2019 Mức tiền thưởng danh hiệu “Chiến sĩ thi đua cấp tỉnh” là 4.470.000 đồng.
Trân trọng!

Score: 0.7551708221435547
----------------------------------------------------------------------------------------------------
Text: Đạt danh hiệu “Chiến sĩ thi đua toàn quốc” sẽ được ưu tiên nâng lương trước hạn?

Answer: Cho hỏi nếu tôi đạt danh hiệu “Chiến sĩ thi đua toàn quốc” thì sẽ được ưu tiên nâng lương trước hạn có đúng không? Căn cứ nào quy định? Xin cảm ơn.

Trả lời:
Căn cứ Điều 75 Nghị định 91/2017/NĐ-CP có quy định:
Cá nhân được tặng dan

<font size = 15 color = 'Green'> Compare with base model</font>

<font size = 5 color = 'Green'> Based on scores</font>

In [16]:
word_embedding_model = models.Transformer(BASE_MODEL)
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)
Base_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [17]:
test_ds = Dataset.from_json(DATA_PATH).train_test_split(test_size=0.1, seed=42)['test']

In [ ]:
# compare model pre-trained and fine-tuned model in test dataset
def compare_base_on_score(pre_train, fine_tune, test_ds, batch_size=32):
    pre_train_score = 0
    fine_tune_score = 0
    source = test_ds['user_question']
    target = test_ds['question']
    pre_train_embeddings_source = pre_train.encode(source, batch_size=batch_size)
    pre_train_embeddings_target = pre_train.encode(target, batch_size=batch_size)
    fine_tune_embeddings_source = fine_tune.encode(source, batch_size=batch_size)
    fine_tune_embeddings_target = fine_tune.encode(target, batch_size=batch_size)
    for i in range(len(test_ds)):
        pre_train_score += util.pytorch_cos_sim(pre_train_embeddings_source[i], pre_train_embeddings_target[i]).item()
        fine_tune_score += util.pytorch_cos_sim(fine_tune_embeddings_source[i], fine_tune_embeddings_target[i]).item()
    avg_pre_train_score = pre_train_score / len(test_ds)
    avg_fine_tune_score = fine_tune_score / len(test_ds)


    print(f"Pre-trained model score: {avg_pre_train_score}")
    print(f"Fine-tuned model score: {avg_fine_tune_score}")

In [ ]:
compare_base_on_score(Base_model, model, test_ds)

In [ ]:
from tqdm import tqdm

# compare model pre-trained and fine-tuned model in test dataset
def compare_base_on_prediction(pre_train, fine_tune, test_ds, batch_size=32, topk = 1):
    pre_train_score = 0
    fine_tune_score = 0
    source = test_ds['user_question']
    target = test_ds['question']
    pre_train_embeddings_source = pre_train.encode(source, batch_size=batch_size)
    pre_train_embeddings_target = pre_train.encode(target, batch_size=batch_size)
    fine_tune_embeddings_source = fine_tune.encode(source, batch_size=batch_size)
    fine_tune_embeddings_target = fine_tune.encode(target, batch_size=batch_size)
    for i in tqdm(range(len(test_ds))):
        # if util.semantic_search(pre_train_embeddings_source[i], pre_train_embeddings_target)[0][0]['corpus_id'] == i:
        #     pre_train_score += 1
        # if util.semantic_search(fine_tune_embeddings_source[i], fine_tune_embeddings_target)[0][0]['corpus_id'] == i:
        #     fine_tune_score += 1
        result_pre_train = util.semantic_search(pre_train_embeddings_source[i], pre_train_embeddings_target)
        result_fine_tune = util.semantic_search(fine_tune_embeddings_source[i], fine_tune_embeddings_target)
        # Consider the last 3 sentences
        for i in range(topk):
            if result_pre_train[0][i]['corpus_id'] == i:
                pre_train_score += 1
                break
        for i in range(topk):
            if result_fine_tune[0][i]['corpus_id'] == i:
                fine_tune_score += 1
                break

    print(f"Pre-trained model score: {pre_train_score}")
    print(f"Fine-tuned model score: {fine_tune_score}")

    # avg_pre_train_score = pre_train_score / len(test_ds)
    # avg_fine_tune_score = fine_tune_score / len(test_ds)


    # print(f"Pre-trained model score: {avg_pre_train_score}")
    # print(f"Fine-tuned model score: {avg_fine_tune_score}")

In [ ]:
compare_base_on_prediction(Base_model, model, test_ds)

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="ThanhRise/intfloat-multilingual-e5-base2023-08-30_16-52-03", revision="f45442111506aab363fa6256e82744fa11ba9d3f")